In [23]:
from collections import defaultdict
import numpy as np
import pandas as pd
from sklearn.metrics import recall_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelBinarizer
from IPython.core.display import display
from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from tqdm import tqdm_notebook

from multi_imbalance.ensemble.SOUPBagging import SOUPBagging
from multi_imbalance.ensemble.mrbbagging import MRBBagging
from multi_imbalance.resampling.SOUP import SOUP
from multi_imbalance.resampling.MDO import MDO
from multi_imbalance.resampling.GlobalCS import GlobalCS

from imblearn.metrics import geometric_mean_score
from imblearn.over_sampling import SMOTE
from multi_imbalance.resampling.spider import SPIDER3

from sklearn.neighbors import KNeighborsClassifier
import warnings
import logging
from multi_imbalance.utils.data import load_arff_datasets
from multi_imbalance.utils.min_int_maj import maj_int_min
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)
warnings.filterwarnings('ignore')

from IPython.display import clear_output
clear_output(wait=True)

In [41]:
# def green_valid_backgroud(s):
#     correct = ['1czysty-cut', '2delikatne-cut', '3mocniej-cut','4delikatne-bezover-cut', 'cmc', 'dermatology', 'new_ecoli','new_vehicle','thyroid-newthyroid']
#     return ['background-color: green' if v in correct else '' for v in list(s.index)]
# 


def bold_max(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_max = s == s.max()
    return ['font-weight: bold' if v else '' for v in is_max]
    
def print_scores(scores, only_read_dt = False, columns=None, base=None):
    df = pd.DataFrame(scores).T
    if only_read_dt:
        df = df.iloc[4:]
    if columns is not None:
        df = df[columns]
    if base is not None:
        df = pd.merge(base,df, left_index=True, right_index=True)
    df2 = df.style.apply(bold_max, axis=1)
    display(df2)

    df.fillna(df.median(), inplace=True)
    display(pd.DataFrame(df.mean().sort_values(ascending=False),columns=['Mean G-mean']))
    display(pd.DataFrame(df.rank(axis=1,ascending=False).mean().sort_values(),columns=['Mean rank']))
# print_scores(scores_knn)

In [25]:
def resample_data(resample, seed, X_train, y_train, no_classes, dataset_name):
    if resample == 'base':
        X_train_resampled, y_train_resampled = X_train, y_train
    elif resample=='soup':
        soup = SOUP(k=3)
        X_train_resampled, y_train_resampled = soup.fit_transform(np.copy(X_train), np.copy(y_train))
    elif resample=='global':
        global_cs = GlobalCS()
        X_train_resampled, y_train_resampled = global_cs.fit_transform(np.copy(X_train), np.copy(y_train), shuffle=False)
    elif resample=='smote':
        smote = SMOTE(random_state=seed)
        X_train_resampled, y_train_resampled = smote.fit_sample(np.copy(X_train), np.copy(y_train))
    elif resample=='mdo':
        mdo = MDO(k=3, k1_frac=0.5, seed=seed)
        X_train_resampled, y_train_resampled = mdo.fit_transform(np.copy(X_train), np.copy(y_train), maj_int_min[dataset_name])
    elif resample=='spider':
        cost = np.ones((no_classes, no_classes))
        np.fill_diagonal(cost, 0)
        clf = SPIDER3(k=5, cost=cost, majority_classes=maj_int_min[dataset_name]['maj'], intermediate_classes=maj_int_min[dataset_name]['int'], minority_classes=maj_int_min[dataset_name]['min'])
        X_train_resampled, y_train_resampled = clf.fit_transform(X_train.astype(np.float64), y_train)
    elif 'soupbg' in resample or 'mrbbag' in resample:
        # SOUP Bagging does it by itself
        X_train_resampled, y_train_resampled = X_train, y_train
    else:
        raise ValueError(f'Bad type{resample}')
    return X_train_resampled, y_train_resampled



def test_resampling(res, dataset_values, dataset_name):
    X, y = dataset_values.data, dataset_values.target

    no_classes = np.unique(y).size
    minority_class = maj_int_min[dataset_name]['min']
    result_data = defaultdict(int)
    run_data = defaultdict(lambda: defaultdict(list)) # {metric: {run_number: [scores]}}
    for i in range(10):
        skf = StratifiedKFold(n_splits=5, shuffle=True,random_state=i)
        for train_index, test_index in skf.split(X, y):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            normalizer = StandardScaler().fit(X_train)

            X_train = normalizer.transform(X_train)
            X_test = normalizer.transform(X_test)
            X_train_resampled, y_train_resampled = resample_data(res, i, X_train, y_train, no_classes, dataset_name)

            # for clf_name in ['knn']:
            for clf_name in ['knn','tree']:
                if clf_name == 'knn':
                    clf = KNeighborsClassifier(n_neighbors=3)
                elif clf_name == 'tree':
                    clf = DecisionTreeClassifier(random_state=i)
                    
                if  'soupbg005' in res:
                    vote_classifier = SOUPBagging(clf, n_classifiers=5)
                    clf = vote_classifier
                elif  'soupbg015' in res:
                    vote_classifier = SOUPBagging(clf, n_classifiers=15)
                    clf = vote_classifier
                elif  'soupbg030' in res:
                    vote_classifier = SOUPBagging(clf, n_classifiers=30)
                    clf = vote_classifier
                elif  'soupbg050' in res:
                    vote_classifier = SOUPBagging(clf, n_classifiers=50)
                    clf = vote_classifier
                elif  'soupbg100' in res:
                    vote_classifier = SOUPBagging(clf, n_classifiers=100)
                    clf = vote_classifier
                # elif res == 'mrbbag005':
                    
                    
                clf.fit(X_train_resampled, y_train_resampled)
                if 'soupbg' in res:
                    for strategy in ['average','optimistic','pessimistic','mixed']:
                        y_pred = clf.predict(X_test, strategy=strategy, maj_int_min=maj_int_min[dataset_name])
                        gmean = geometric_mean_score(y_test, y_pred, correction=0.001)
                        minority_gmean = geometric_mean_score(y_test, y_pred,labels=minority_class, correction=0.001)
                        avg_acc = np.mean(recall_score(y_test, y_pred, average=None))
                        run_data['g_mean_{}_{}'.format(clf_name, strategy)][str(i)].append(gmean)
                        run_data['g_mean_{}_minority_{}'.format(clf_name, strategy)][str(i)].append(minority_gmean)
                else:
                    y_pred = clf.predict(X_test)
                    gmean = geometric_mean_score(y_test, y_pred, correction=0.001)
                    minority_gmean = geometric_mean_score(y_test, y_pred,labels=minority_class, correction=0.001)
                    avg_acc = np.mean(recall_score(y_test, y_pred, average=None))
                    run_data['g_mean_{}'.format(clf_name)][str(i)].append(gmean)
                    run_data['g_mean_{}_minority'.format(clf_name)][str(i)].append(minority_gmean)
                # run_data['avg_acc_{}'.format(clf_name)][str(i)].append(avg_acc)
    
    def get_score_from_metric(run_data, metric):
        runs = run_data[metric]
        runs_scores_list = list(runs.values()) #[[one run k-foledscores],[..]]
        result = np.mean(list(map(np.mean, runs_scores_list)))
        return result
            
    for metric_name, metric_values in run_data.items():
        result_data[metric_name] = get_score_from_metric(run_data, metric_name)
        
    return result_data


def provide_test_and_get_scores(dataset, clf_res):
    scores = defaultdict(lambda: defaultdict(dict))
    for dataset_name, dataset_values in tqdm_notebook(datasets.items(),total=len(datasets), desc='1st loop'):
        for resample in clf_res_names:
            result_data = test_resampling(resample, dataset_values, dataset_name)
            for key in result_data:
                scores[key][dataset_name][resample] = round(result_data[key],4)
    return scores

clf_res_names =['base','soup','soupbg005','soupbg015','soupbg030','soupbg050','soupbg100']
datasets = load_arff_datasets()
scores = provide_test_and_get_scores(datasets, clf_res_names)


### Kfold - 5, powtórzone 10 razy
#### Drzewo, głosowanie: average, soup k = 3, miara Gmean

In [42]:
# scores
base = pd.DataFrame(scores['g_mean_tree']).T
columns = [i for i in clf_res_names if 'soupbg' in i]
print_scores(scores['g_mean_tree_average'], columns=columns, base=base)


,base,soup,soupbg005,soupbg015,soupbg030,soupbg050,soupbg100
1czysty-cut,0.9391,0.9574,0.9374,0.9374,0.9487,0.9516,0.9553
2delikatne-cut,0.6976,0.775,0.7739,0.7739,0.7774,0.7908,0.7927
3mocniej-cut,0.492,0.5664,0.5553,0.5553,0.5736,0.5776,0.5907
4delikatne-bezover-cut,0.7711,0.8749,0.8728,0.8728,0.8785,0.8785,0.8816
balance-scale,0.1562,0.5774,0.5248,0.5248,0.5365,0.5592,0.5604
car,0.9271,0.93,0.9109,0.9109,0.9248,0.9352,0.9458
cleveland,0.1188,0.1116,0.1125,0.1125,0.1315,0.12,0.1314
cleveland_v2,0.133,0.144,0.1722,0.1722,0.1481,0.1117,0.1199
cmc,0.4424,0.4585,0.46,0.4619,0.4832,0.4924,0.5018
dermatology,0.9274,0.9407,0.9378,0.9378,0.9533,0.9557,0.9579


,Mean G-mean
soupbg100,0.663305
soupbg050,0.657284
soupbg030,0.655516
soup,0.651337
soupbg015,0.631374
soupbg005,0.630968
base,0.594337


,Mean rank
soupbg100,2.000000
soupbg050,2.657895
soupbg030,3.026316
soup,3.473684
base,5.526316
soupbg005,5.657895
soupbg015,5.657895


#### Drzewo, głosowanie: average, soup k = 3, miara Gmean ale tylko dla klas mniejszościowych

In [45]:
base = pd.DataFrame(scores['g_mean_tree_minority']).T
columns = [i for i in clf_res_names if 'soupbg' in i]
print_scores(scores['g_mean_tree_minority_average'], columns=columns, base=base)



,base,soup,soupbg005,soupbg015,soupbg030,soupbg050,soupbg100
1czysty-cut,0.9142,0.9542,0.9092,0.9092,0.9383,0.9442,0.9525
2delikatne-cut,0.5367,0.67,0.665,0.665,0.6692,0.7,0.6867
3mocniej-cut,0.3133,0.4292,0.4142,0.4142,0.4333,0.4308,0.4333
4delikatne-bezover-cut,0.6858,0.8833,0.875,0.875,0.8858,0.8775,0.8792
balance-scale,0.0232,0.3987,0.2963,0.2963,0.2776,0.3011,0.2798
car,0.8989,0.9161,0.8825,0.8825,0.8961,0.9117,0.9301
cleveland,0.0812,0.0781,0.079,0.079,0.0964,0.0831,0.0927
cleveland_v2,0.0891,0.101,0.1259,0.1259,0.1001,0.065,0.0719
cmc,0.364,0.4384,0.4364,0.4358,0.4486,0.4603,0.472
dermatology,0.91,0.925,0.935,0.935,0.95,0.945,0.97


,Mean G-mean
soup,0.624689
soupbg100,0.623026
soupbg030,0.620221
soupbg050,0.619053
soupbg015,0.597774
soupbg005,0.597716
base,0.530068


,Mean rank
soupbg100,2.552632
soup,2.894737
soupbg030,2.973684
soupbg050,3.210526
soupbg005,5.131579
soupbg015,5.236842
base,6.000000


#### kNN - 3, miara Gmean

In [43]:
base = pd.DataFrame(scores['g_mean_knn']).T
columns = [i for i in clf_res_names if 'soupbg' in i]
print_scores(scores['g_mean_knn_average'], columns=columns, base=base)



,base,soup,soupbg005,soupbg015,soupbg030,soupbg050,soupbg100
1czysty-cut,0.961,0.9661,0.9571,0.9605,0.9638,0.9663,0.9654
2delikatne-cut,0.6811,0.7759,0.7823,0.7824,0.7858,0.782,0.7835
3mocniej-cut,0.4345,0.5387,0.5278,0.5342,0.5405,0.5469,0.5502
4delikatne-bezover-cut,0.7867,0.8739,0.8741,0.8775,0.8799,0.8797,0.8806
balance-scale,0.1184,0.6068,0.5607,0.5504,0.553,0.5425,0.5448
car,0.3173,0.5013,0.4986,0.5378,0.541,0.5344,0.5276
cleveland,0.0537,0.1053,0.1009,0.0979,0.1013,0.1085,0.0999
cleveland_v2,0.0437,0.1125,0.1554,0.1547,0.0956,0.0992,0.1128
cmc,0.4167,0.4527,0.4203,0.4308,0.4431,0.4471,0.4505
dermatology,0.9416,0.925,0.9124,0.9232,0.9396,0.9377,0.9387


,Mean G-mean
soupbg100,0.620600
soup,0.617021
soupbg050,0.614963
soupbg030,0.611189
soupbg015,0.607921
soupbg005,0.599537
base,0.513021


,Mean rank
soupbg100,2.315789
soupbg050,3.000000
soup,3.315789
soupbg030,3.473684
soupbg015,4.473684
soupbg005,5.421053
base,6.000000


#### kNN - 3, miara Gmean dla klas mniejszościowych



In [47]:
base = pd.DataFrame(scores['g_mean_knn_minority']).T
columns = [i for i in clf_res_names if 'soupbg' in i]
print_scores(scores['g_mean_knn_minority_average'], columns=columns, base=base)



,base,soup,soupbg005,soupbg015,soupbg030,soupbg050,soupbg100
1czysty-cut,0.9383,0.9992,0.98,0.9817,0.9883,0.995,0.9958
2delikatne-cut,0.49,0.7008,0.7067,0.715,0.7275,0.715,0.715
3mocniej-cut,0.2342,0.4442,0.4067,0.4292,0.4475,0.4617,0.4575
4delikatne-bezover-cut,0.6858,0.9158,0.9008,0.9042,0.92,0.9158,0.9167
balance-scale,0.0091,0.4833,0.3542,0.3153,0.3334,0.3065,0.3027
car,0.1928,0.4471,0.3898,0.4343,0.44,0.4393,0.4322
cleveland,0.0284,0.072,0.0697,0.0667,0.0666,0.0739,0.0658
cleveland_v2,0.0174,0.0688,0.1088,0.1088,0.0517,0.0538,0.0683
cmc,0.3387,0.4673,0.4474,0.4795,0.5017,0.5093,0.5222
dermatology,0.95,1,0.96,0.965,0.975,0.975,0.99


,Mean G-mean
soup,0.613332
soupbg100,0.605247
soupbg050,0.597400
soupbg030,0.595795
soupbg015,0.587221
soupbg005,0.578174
base,0.426316


,Mean rank
soup,2.447368
soupbg100,2.947368
soupbg050,3.052632
soupbg030,3.394737
soupbg015,4.342105
soupbg005,4.815789
base,7.000000


#### Porównanie różnych głosowań dla knn - 3 i SOUP Bagging - 100 klasyfikatorów, miara Gmean

In [65]:
# scores
base = pd.DataFrame(scores['g_mean_knn']).T
metrices = [k for k in scores if 'knn' in k and 'minority' not in k][1:]
# metrices
for metric in metrices:
    temp_df = pd.DataFrame(pd.DataFrame(scores[metric]).T['soupbg100'])
    temp_df.columns = [metric.split('_')[-1]]
    base = pd.merge(base, temp_df, left_index=True, right_index=True)
print_scores(base.T)

,base,soup,average,optimistic,pessimistic,mixed
1czysty-cut,0.961,0.9661,0.9654,0.9556,0.9555,0.953
2delikatne-cut,0.6811,0.7759,0.7835,0.7312,0.6353,0.7635
3mocniej-cut,0.4345,0.5387,0.5502,0.4674,0.3253,0.5062
4delikatne-bezover-cut,0.7867,0.8739,0.8806,0.8299,0.8306,0.8784
balance-scale,0.1184,0.6068,0.5448,0.6306,0.6913,0.6382
car,0.3173,0.5013,0.5276,0.2382,0.0697,0.5543
cleveland,0.0537,0.1053,0.0999,0.0562,0.01,0.0602
cleveland_v2,0.0437,0.1125,0.1128,0.0545,0.0075,0.0631
cmc,0.4167,0.4527,0.4505,0.3917,0.2831,0.2592
dermatology,0.9416,0.925,0.9387,0.92,0.8602,0.882


,Mean G-mean
average,0.620600
soup,0.617021
mixed,0.534079
base,0.513021
optimistic,0.507074
pessimistic,0.428747


,Mean rank
average,1.684211
soup,2.052632
mixed,3.736842
base,3.789474
optimistic,4.342105
pessimistic,5.394737


#### Porównanie różnych głosowań dla knn - 3 i SOUP Bagging - 100 klasyfikatorów, Gmean dla klas mniejszościowych


In [66]:
base = pd.DataFrame(scores['g_mean_knn']).T
metrices = [k for k in scores if 'knn' in k and 'minority' in k][1:]
# metrices
for metric in metrices:
    temp_df = pd.DataFrame(pd.DataFrame(scores[metric]).T['soupbg100'])
    temp_df.columns = [metric.split('_')[-1]]
    base = pd.merge(base, temp_df, left_index=True, right_index=True)
print_scores(base.T)



,base,soup,average,optimistic,pessimistic,mixed
1czysty-cut,0.961,0.9661,0.9958,0.925,0.9225,1
2delikatne-cut,0.6811,0.7759,0.715,0.7192,0.4683,0.9142
3mocniej-cut,0.4345,0.5387,0.4575,0.4425,0.155,0.785
4delikatne-bezover-cut,0.7867,0.8739,0.9167,0.7342,0.7342,0.9967
balance-scale,0.1184,0.6068,0.3027,0.684,0.9553,0.9778
car,0.3173,0.5013,0.4322,0.1168,0.0158,0.5922
cleveland,0.0537,0.1053,0.0658,0.0306,0.0034,0.0412
cleveland_v2,0.0437,0.1125,0.0683,0.0236,0.0016,0.0334
cmc,0.4167,0.4527,0.5222,0.4963,0.1666,0.873
dermatology,0.9416,0.925,0.99,0.89,0.875,1


,Mean G-mean
mixed,0.750332
soup,0.617021
average,0.605247
base,0.513021
optimistic,0.469253
pessimistic,0.404400


,Mean rank
mixed,1.421053
average,2.526316
soup,2.684211
base,3.947368
optimistic,4.684211
pessimistic,5.736842
